<a href="https://colab.research.google.com/github/RazanHL/ML-peer-review/blob/main/BERT_multilabel_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Requerments

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 15.4 MB/s 
     |████████████████████████████████| 6.6 MB 61.5 MB/s 
     |████████████████████████████████| 163 kB 71.6 MB/s 


In [ ]:
from tqdm import tqdm
from sklearn.metrics import (accuracy_score, classification_report,multilabel_confusion_matrix,
                             f1_score, precision_score, recall_score)

In [ ]:
import torch
from datetime import datetime
import pandas as pd
import numpy as np
import shutil
import sys

# Dataset pre-processing:

In [ ]:
articles = joblib.load("/content/drive/MyDrive/MWT folder/MWT master/backups/bert_dataset.pkl")
articles.head()

,Article_ID,folder,ar_en,Country,Title,Author,Author_certificate,Author_specification,work,co_Author,...,Rev1_score,Rev2_score,Rev3_score,Score,Volume,Issue,Result,Date_of_editor_decision,label,full_text_clean
0,1,14-1,ar,Syria,تأثير طريقة الزراعة وعمق الحراثة في نمو الشوند...,فادي عباس,الدكتور,انتاج محاصيل (شوندر سكري),الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,محمود السيدو,...,2.857143,3.142857,3.785714286,3.261904762,1,1,مقبول,NaN,0,تأثير طريقة الزراعة وعمق الحراثة في نمو الشوند...
1,3,14-3,ar,Syria,التأثير الطارد لمركب الكلس المكروني (Surround)...,مازن بوفاعور,الدكتور,وقاية النبات,الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,"فيلي خاريزانوفا, ناريمان الزغبي",...,3.357143,3.357143,2,2.904761905,1,1,مقبول,NaN,1,التأثير الطارد لمركب الكلس المكروني على وضع ال...
2,4,14-4,ar,Syria,تأثير طريقة الري بالتنقيط (مغطى ومكشوف) في الا...,عبدالكريم جردي,المهندس,ري محاصيل حقلية,الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,"بسام عودة, هيثم العبدالله, طلال العبدو, بشرى خزام",...,3.571429,3.785714,3,3.452380952,2,2,مقبول,27_11_2014,2,تأثير طريقة الري بالتنقيط مغطى ومكشوف في الاست...
3,5,14-5,ar,Syria,تغيرات مجتمع حشرة بسيلا الأجاص Cacopsylla pyri...,بسام عودة,الدكتور,وقاية نبات (حشرات),الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,"وجيه قسيس, رندة أبو طارة",...,3.428571,2.357143,NaN,2.892857143,2,1,مقبول,NaN,1,تأثير المعقد الحيوي على مجتمعات حشرة بسيلا الأ...
4,6,14-6,ar,Syria,تأثير سبعة أصول من الحمضيات في مواصفات النمو و...,علاء ابراهيم,المهندس,علوم البستنة,الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,"إشراق سليمان علي, رائد صبيح, رفيق علي عبود, فا...",...,3.071429,2.928571,2.642857143,2.880952381,1,1,مقبول,NaN,2,تأثير سبعة أصول من الحمضيات في مواصفات النمو و...


In [ ]:
# Transforming Multi-class --> Multi-label case
labels = articles.Scope.replace({
    'اقتصاد زراعي': "Agricultural_Economics",
    'بستنة': "Horticulture",
    'محاصيل حقلية': "Crop_production",
    'موارد': "Natural_Resources",
    'وقاية نبات': "Plant_Protection",
    'تقانات حيوية': "Agriculture_Biotechnology",
    'أغذية': "Food_Technology",
    'إنتاج حيواني': "Livestock_production",
    'بيئة وحراج': "Environmental_Sciences"
})

In [ ]:
multi_labels = pd.get_dummies(labels)
multi_labels

,Agricultural_Economics,Agriculture_Biotechnology,Crop_production,Environmental_Sciences,Food_Technology,Horticulture,Livestock_production,Natural_Resources,Plant_Protection
0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
993,0,0,0,0,0,1,0,0,0
994,0,0,0,0,1,0,0,0,0
995,0,0,0,0,0,0,1,0,0
996,1,0,0,0,0,0,0,0,0


In [ ]:
# Adding multi-label columns to main dataframe
articles = articles.merge(multi_labels, left_index=True, right_index=True)
articles

In [ ]:
articles.to_csv('/content/drive/MyDrive/articles7_9_22.csv', header=True)

In [ ]:
# Manual label correction then importing transformed dataset
articles = pd.read_csv('/content/drive/MyDrive/articles9_9_22.csv')
articles.head()

,Unnamed: 0,Article_ID,folder,ar_en,Country,Title,Author,Author_certificate,Author_specification,work,...,abstract,Agricultural_Economics,Agriculture_Biotechnology,Crop_production,Environmental_Sciences,Food_Technology,Horticulture,Livestock_production,Natural_Resources,Plant_Protection
0,0,1,14-1,ar,Syria,تأثير طريقة الزراعة وعمق الحراثة في نمو الشوند...,فادي عباس,الدكتور,انتاج محاصيل (شوندر سكري),الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,...,تأثير طريقة الزراعة وعمق الحراثة في نمو الشوند...,0,0,1,0,0,0,0,0,0
1,1,3,14-3,ar,Syria,التأثير الطارد لمركب الكلس المكروني (Surround)...,مازن بوفاعور,الدكتور,وقاية النبات,الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,...,التأثير الطارد لمركب الكلس المكروني على وضع ال...,0,0,0,0,0,0,0,0,1
2,2,4,14-4,ar,Syria,تأثير طريقة الري بالتنقيط (مغطى ومكشوف) في الا...,عبدالكريم جردي,المهندس,ري محاصيل حقلية,الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,...,تأثير طريقة الري بالتنقيط مغطى ومكشوف في الاست...,0,0,0,0,0,1,0,1,0
3,3,5,14-5,ar,Syria,تغيرات مجتمع حشرة بسيلا الأجاص Cacopsylla pyri...,بسام عودة,الدكتور,وقاية نبات (حشرات),الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,...,تأثير المعقد الحيوي على مجتمعات حشرة بسيلا الأ...,0,0,0,0,0,0,0,0,1
4,4,6,14-6,ar,Syria,تأثير سبعة أصول من الحمضيات في مواصفات النمو و...,علاء ابراهيم,المهندس,علوم البستنة,الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,...,تأثير سبعة أصول من الحمضيات في مواصفات النمو و...,0,0,0,0,0,1,0,0,0


In [ ]:
# Splitting long text into chunks of 150 words each
def get_split(text1):
  l_total = []
  l_parcial = []
  if len(text1.split())//150 >0:
    n = len(text1.split())//150
  else:
    n = 1
  for w in range(n):
    if w == 0:
      l_parcial = text1.split()[:200]
      l_total.append(" ".join(l_parcial))
    else:
      l_parcial = text1.split()[w*150:w*150 + 200]
      l_total.append(" ".join(l_parcial))
  return l_total

In [ ]:
articles['split_txt'] = articles['full_text_clean'].apply(get_split)

In [ ]:
# Train- Test split
train_size = int(998*0.8)
train_dataset = articles.sample(train_size, random_state= 45)
val_dataset = articles.drop(axis=0, index=train_dataset.index)


In [ ]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 798 entries, 657 to 29
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   abstract                   798 non-null    object
 1   Agricultural_Economics     798 non-null    int64 
 2   Agriculture_Biotechnology  798 non-null    int64 
 3   Crop_production            798 non-null    int64 
 4   Environmental_Sciences     798 non-null    int64 
 5   Food_Technology            798 non-null    int64 
 6   Horticulture               798 non-null    int64 
 7   Livestock_production       798 non-null    int64 
 8   Natural_Resources          798 non-null    int64 
 9   Plant_Protection           798 non-null    int64 
 10  split_txt                  798 non-null    object
dtypes: int64(9), object(2)
memory usage: 74.8+ KB


In [ ]:
train_l = []
label1, label2, label3, label4, label5, label6, label7, label8, label9 = [], [],[],[],[],[],[],[],[]
index_l =[]
for idx,row in train_dataset.iterrows():
  for l in row['split_txt']:
    train_l.append(l)
    label1.append(row['Agricultural_Economics'])
    label2.append(row['Agriculture_Biotechnology'])
    label3.append(row['Crop_production'])
    label4.append(row['Environmental_Sciences'])
    label5.append(row['Food_Technology'])
    label6.append(row['Horticulture'])
    label7.append(row['Livestock_production'])
    label8.append(row['Natural_Resources'])
    label9.append(row['Plant_Protection'])
    index_l.append(idx)
len(train_l), len(index_l)

(13206, 13206)

In [ ]:
splited_train = pd.DataFrame({"ids": index_l, "sp_text": train_l,
                              "Agricultural_Economics": label1,
                              "Agriculture_Biotechnology": label2,
                              "Crop_production": label3,
                              "Environmental_Sciences": label4,
                              "Food_Technology": label5,
                              "Horticulture": label6,
                              "Livestock_production": label7,
                              "Natural_Resources": label8,
                              "Plant_Protection": label9})
splited_train.head()

,ids,sp_text,Agricultural_Economics,Agriculture_Biotechnology,Crop_production,Environmental_Sciences,Food_Technology,Horticulture,Livestock_production,Natural_Resources,Plant_Protection
0,657,تسجيل جديد للعثة المسببة للتدرن على الأجاص من ...,0,0,0,0,0,0,0,0,1
1,467,تشخيص وكشف فيروس التهاب القصبات المعدي استخدام...,0,1,0,0,0,0,1,0,0
2,467,مع مخبر ال في كلية الطب البيطري بحماه بينت نتا...,0,1,0,0,0,0,1,0,0
3,467,أكبر انحراف في تسلسل الأحماض الأمينية في الموا...,0,1,0,0,0,0,1,0,0
4,467,للحصول على جيد النوعية لذلك يتوجب عزل الفيروس ...,0,1,0,0,0,0,1,0,0


In [ ]:
splited_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13206 entries, 0 to 13205
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ids                        13206 non-null  int64 
 1   sp_text                    13206 non-null  object
 2   Agricultural_Economics     13206 non-null  int64 
 3   Agriculture_Biotechnology  13206 non-null  int64 
 4   Crop_production            13206 non-null  int64 
 5   Environmental_Sciences     13206 non-null  int64 
 6   Food_Technology            13206 non-null  int64 
 7   Horticulture               13206 non-null  int64 
 8   Livestock_production       13206 non-null  int64 
 9   Natural_Resources          13206 non-null  int64 
 10  Plant_Protection           13206 non-null  int64 
dtypes: int64(10), object(1)
memory usage: 1.1+ MB


In [ ]:
def print_catigories(my_dataset):
  print(my_dataset.Agricultural_Economics.value_counts()[1], 'Agricultural_Economics')
  print(my_dataset.Agriculture_Biotechnology.value_counts()[1], 'Agriculture_Biotechnology')
  print(my_dataset.Crop_production.value_counts()[1],'Crop_production')
  print(my_dataset.Environmental_Sciences.value_counts()[1], 'Environmental_Sciences')
  print(my_dataset.Food_Technology.value_counts()[1], 'Food_Technolog')
  print(my_dataset.Horticulture.value_counts()[1], 'Horticulture')
  print(my_dataset.Livestock_production.value_counts()[1], 'Livestock_production')
  print(my_dataset.Natural_Resources.value_counts()[1], 'Natural_Resources')
  print(my_dataset.Plant_Protection.value_counts()[1], 'Plant_Protection')

print_catigories(splited_train)
# splited_train.groupby(['Agricultural_Economics']).sum().describe()

1916 Agricultural_Economics
737 Agriculture_Biotechnology
2954 Crop_production
1772 Environmental_Sciences
1011 Food_Technolog
2804 Horticulture
1361 Livestock_production
3280 Natural_Resources
1612 Plant_Protection


# Preparing Oversampled Data

In [ ]:
articles =pd.read_csv('/content/drive/MyDrive/oversampled_balanced.csv')


In [ ]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28646 entries, 0 to 28645
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Unnamed: 0                 28646 non-null  int64 
 1   ids                        28646 non-null  int64 
 2   sp_text                    28646 non-null  object
 3   Agricultural_Economics     28646 non-null  int64 
 4   Agriculture_Biotechnology  28646 non-null  int64 
 5   Crop_production            28646 non-null  int64 
 6   Environmental_Sciences     28646 non-null  int64 
 7   Food_Technology            28646 non-null  int64 
 8   Horticulture               28646 non-null  int64 
 9   Livestock_production       28646 non-null  int64 
 10  Natural_Resources          28646 non-null  int64 
 11  Plant_Protection           28646 non-null  int64 
dtypes: int64(11), object(1)
memory usage: 2.6+ MB


In [ ]:
articles = articles.sample(frac= 1, random_state= 42).reset_index(drop=True)


In [ ]:
train_dataset = articles.sample(frac=0.8 , random_state= 45)
val_dataset = articles.drop(axis=0, index=train_dataset.index)
train_dataset.shape, val_dataset.shape

((22917, 12), (5729, 12))

In [ ]:
train_dataset = train_dataset.reset_index(drop=True)
val_dataset = val_dataset.reset_index(drop=True)

# Training Model:

In [ ]:
target_list =['Agricultural_Economics',	'Agriculture_Biotechnology',	'Crop_production',
              'Environmental_Sciences',	'Food_Technology',	'Horticulture',	'Livestock_production',
              'Natural_Resources',	'Plant_Protection']
len(target_list)

9

In [ ]:
# hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-05

In [ ]:
from transformers import AutoModel, AutoTokenizer
model_name = "aubmindlab/bert-base-arabertv2"
# arabert_prep = ArabertPreprocessor(model_name=model_name)
arabert_model = AutoModel.from_pretrained(model_name, return_dict=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
class ClassificationDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.txt = self.df['sp_text']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.txt)

    def __getitem__(self, index):
        txt = str(self.txt[index])
        txt = " ".join(txt.split())

        inputs = self.tokenizer.encode_plus(
            txt,
            None, # to tokenize only one sentence not 2
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [ ]:
train_data = ClassificationDataset(train_dataset, tokenizer, MAX_LEN)
valid_data = ClassificationDataset(val_dataset, tokenizer, MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_data,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_data,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min  #.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.model = arabert_model
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 9)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output


In [ ]:
model = BERTClass()
model.to(device)

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =model.parameters(), lr=LEARNING_RATE)


In [ ]:
def accuracy_fn(targets, outputs):
  y_pred=[]
  for sample in outputs:
    y_pred.append([1 if i>=0.5 else 0 for i in sample] )
  y_pred = np.array(y_pred)
  y_labels = np.array(targets)
  report = classification_report(y_labels, y_pred,target_names=target_list, zero_division=1)
  acc = accuracy_score(y_labels, y_pred)
  f1 = f1_score(y_labels, y_pred, average='macro', zero_division=1)
  per = precision_score(y_labels, y_pred, average='macro', zero_division=1)
  recall = recall_score(y_labels, y_pred,average='macro', zero_division=1)
  # cm = multilabel_confusion_matrix(y_labels, y_pred, samplewise=True)
  return (report, acc, f1, per, recall)


In [ ]:

def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf


  for epoch in range(1, n_epochs+1):
    start_training_time = datetime.now()
    train_loss = 0
    valid_loss = 0
    val_targets=[]
    val_outputs=[]

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    print('Training started at: {}'.format(start_training_time))
    for batch_idx, data in enumerate(training_loader):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)

    print('############# Epoch {}: Training End     #############'.format(epoch))
    end_training_time = datetime.now()
    print('Training ended after: {}'.format(end_training_time - start_training_time))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    start_validation_time = datetime.now()
    print('Validation started at: {}'.format(start_training_time))

    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
      # report, acc, f1, per, recall, cm = accuracy_fn(val_targets, val_outputs)


      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      ## save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss
    report, acc, f1, per, recall = accuracy_fn(val_targets, val_outputs)
    print(report)
    print('Model Accuracy: ', acc)
    print('Total F1_score: ', f1)
    print('Total Percision', per)
    print('Total Recall', recall)
    end_validation_time = datetime.now()
    print('\n Validation ended after: {}'.format(end_validation_time - start_validation_time))

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [ ]:
# 28/9/2022
ckpt_path = "/content/drive/MyDrive/araBERTv2/test2/curr_ckpt"
best_model_path = "/content/drive/MyDrive/araBERTv2/test2/best_model.pt"
trained_model = train_model(epoch_num, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

############# Epoch 1: Training Start   #############
Training started at: 2022-09-29 05:05:43.043007
############# Epoch 1: Training End     #############
Training ended after: 0:16:57.063237
############# Epoch 1: Validation Start   #############
Validation started at: 2022-09-29 05:05:43.043007
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.000039 	Average Validation Loss: 0.000287
Validation loss decreased (inf --> 0.000287).  Saving model ...
                           precision    recall  f1-score   support

   Agricultural_Economics       0.99      0.98      0.98       828
Agriculture_Biotechnology       0.97      0.98      0.98       803
          Crop_production       0.94      0.92      0.93       862
   Environmental_Sciences       0.97      0.88      0.93       795
          Food_Technology       0.98      0.98      0.98       831
             Horticulture       0.92      0.89      0.90       789
     Livestock_production       0

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/araBERTv2/arabert_classifier.pt')

In [ ]:
# loading trained model
model, optimizer, epoch_num, min_valid_loss = load_ckp('/content/drive/MyDrive/araBERTv2/curr_ckpt.ckpt', model,  optimizer)

# Loading best model

In [ ]:
# device = torch.device('cpu')
bert_model = BERTClass()
bert_model.load_state_dict(torch.load('/content/drive/MyDrive/araBERTv2/test2/arabert_classifier.pt')) # , map_location=device)
bert_model.eval()

In [ ]:
state_dict = torch.load('/content/drive/MyDrive/araBERTv2/arabert_classifier.pt')
print(state_dict.keys())

odict_keys(['model.embeddings.position_ids', 'model.embeddings.word_embeddings.weight', 'model.embeddings.position_embeddings.weight', 'model.embeddings.token_type_embeddings.weight', 'model.embeddings.LayerNorm.weight', 'model.embeddings.LayerNorm.bias', 'model.encoder.layer.0.attention.self.query.weight', 'model.encoder.layer.0.attention.self.query.bias', 'model.encoder.layer.0.attention.self.key.weight', 'model.encoder.layer.0.attention.self.key.bias', 'model.encoder.layer.0.attention.self.value.weight', 'model.encoder.layer.0.attention.self.value.bias', 'model.encoder.layer.0.attention.output.dense.weight', 'model.encoder.layer.0.attention.output.dense.bias', 'model.encoder.layer.0.attention.output.LayerNorm.weight', 'model.encoder.layer.0.attention.output.LayerNorm.bias', 'model.encoder.layer.0.intermediate.dense.weight', 'model.encoder.layer.0.intermediate.dense.bias', 'model.encoder.layer.0.output.dense.weight', 'model.encoder.layer.0.output.dense.bias', 'model.encoder.layer.0.o

In [ ]:
# Prediction fuction
def arabert_predict(sample, this_model):
  encodings = tokenizer.encode_plus(
    sample,
    None,
    add_special_tokens=True,
    max_length= 256,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
  )
  this_model.eval()
  with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = this_model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    # print(train_dataset.columns[1:].to_list()[(np.argmax(final_output, axis=1)).astype(int)])
    predictions = []
    for idx, label in enumerate(target_list):
      if (final_output[0][idx] > 0.5):
        predictions.append((label,round(final_output[0][idx]*100, 2)))  # round(preds[idx]*100, 2
    return (predictions)

In [ ]:
sample = articles.sp_text[770]
sample

In [ ]:
arabert_predict(sample, bert_model)

[('Horticulture', 99.71)]

In [ ]:
sample2

In [ ]:
arabert_predict(sample2, bert_model)

[('Plant_Protection', 99.89)]

In [ ]:
sample3 = articles.sp_text[18000]
articles.iloc[18000]

In [ ]:
arabert_predict(sample3, bert_model)

[('Agriculture_Biotechnology', 99.48), ('Food_Technology', 99.73)]

# Model Evaluation:

In [ ]:
y_pred = []
y_test = splited_test[target_list].values


In [ ]:
len(y_pred)


3407

In [ ]:
for txt in splited_test.sp_text:
  y_pred.append(arabert_predict(txt, model))

In [ ]:
report, acc, f1, per, recall = accuracy_fn(y_test, y_pred)
print(report)
print('Model Accuracy: ', acc)
print('Total F1_score: ', f1)
print('Total Percision', per)
print('Total Recall', recall)

                           precision    recall  f1-score   support

   Agricultural_Economics       0.99      0.97      0.98       501
Agriculture_Biotechnology       0.88      0.85      0.87       153
          Crop_production       0.94      0.78      0.85       596
   Environmental_Sciences       0.92      0.91      0.91       486
          Food_Technology       0.93      0.94      0.94       464
             Horticulture       0.86      0.94      0.90       714
     Livestock_production       0.94      0.97      0.96       219
        Natural_Resources       0.88      0.95      0.91       786
         Plant_Protection       0.96      0.92      0.94       471

                micro avg       0.92      0.92      0.92      4390
                macro avg       0.92      0.91      0.92      4390
             weighted avg       0.92      0.92      0.92      4390
              samples avg       0.93      0.93      0.92      4390

Model Accuracy:  0.8649838567654828
Total F1_score:  0.9178